<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_generate_color_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf

--2021-10-03 12:36:58--  https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf [following]
--2021-10-03 12:36:58--  https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92920 (91K) [application/octet-stream]
Saving to: ‘Dosis-Bold.ttf’

Dosis-Bold.ttf      100%[===================>]  90.74K  --.-KB/s    in 0.01s   

2021-10-03 12:36:58 (6.28 MB/s) - ‘Dosis-Bold.ttf’ saved [92920/92920]



In [2]:
import os
import numpy as np
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import requests
from io import BytesIO
from google.colab import files
import shutil
from datetime import date

colors={'w':(255, 255, 255, 255),
    'k':(0, 0, 0, 255),   #black
    'r':(255, 0, 0, 255),    #red
    '1':(255, 0, 221, 255),  #pink
    'b':(0,0,255,255), #blue
    '2':(0,255,69,255),
    '3':(128,0,128,255), #purple
    'o':(255,165,0,255), #orange
    'y':(255,255,0,255)} #yellow

text="""after i escaped the
dark %okk%and dangerous
tunnel %yk1%in the%3oy% last%wkk% part""" 

rootname='subtitles'+date.today().strftime('%d%b%y')

root='./'+rootname
if not os.path.exists(root):
    os.makedirs(root)
imw,imh=1920,250

fnt = ImageFont.truetype("Dosis-Bold.ttf", 115)

In [3]:
text_lines=text.split('\n')
for i,text_line in enumerate(text_lines):
    z = re.match(r"%\w{3}%", text_line)
    if not z:
        text_lines[i]='%wkk%'+text_line
    

In [4]:
i=0
w_start=0
filenames=[]
for text_line in text_lines:
    z = re.split(r"(%\w{3}%)", text_line)
    color_strings = z[1::2]
    text_strings = z[2::2]

    full_text_line = ''.join(text_strings)
    w_end = w_start + len(full_text_line.split())

    filename = f'a_{i:04d}_{w_start:04d}_{w_end:04d}.png'
    fullname=root+ '/' + filename
    i+=1
    w_start = w_end

    im = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d = ImageDraw.Draw(im)
    imface = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d2 = ImageDraw.Draw(imface)




    w,h=d.textsize(full_text_line,fnt)
    x,y = (imw-w)//2, (imh-h)//2
    prior_text=''
    for color_string, text_string in zip(color_strings,text_strings):
        font_c = color_string[1]
        outline_c = color_string[2]
        shadow_c = color_string[3]

        d.text((x,y), text_string, font=fnt, fill=colors[shadow_c])
        d2.text((x,y), text_string, font=fnt, fill=colors[font_c], stroke_width=7, stroke_fill=colors[outline_c])

        prior_text+=text_string
        x+=d.textsize(text_string,fnt)[0]

    im = im.filter(ImageFilter.GaussianBlur(radius=30))
    imnp_raw=np.asarray(im)
    imnp=imnp_raw.copy()
    imnp[:,:,3]=(imnp_raw[:,:,3]*(255.0/imnp_raw[:,:,3].max())).astype(np.uint8)
    imfacenp = np.asarray(imface)
    blank = np.zeros((imh,imw,4),dtype=np.uint8)
    compnp=np.where((imnp[:,:,3]>0)[:,:,None], imnp, blank)
    compnp=np.where((imfacenp[:,:,3]>200)[:,:,None], imfacenp, compnp)

    Image.fromarray(compnp,'RGBA').save(fullname)
    filenames.append(filename)

In [5]:
shutil.make_archive(rootname,'zip',root_dir =root)

'/content/subtitles03Oct21.zip'

In [6]:
duration=5000
path='D:/Downloads/'+rootname

In [26]:
timeline_start=f"""<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE xmeml>
<xmeml version="5">
    <sequence>
        <name>Timeline 1 (Resolve)</name>
        <duration>{duration}</duration>
        <rate>
            <timebase>60</timebase>
            <ntsc>FALSE</ntsc>
        </rate>
        <in>-1</in>
        <out>-1</out>
        <timecode>
            <string>01:00:00:00</string>
            <frame>216000</frame>
            <displayformat>NDF</displayformat>
            <rate>
                <timebase>60</timebase>
                <ntsc>FALSE</ntsc>
            </rate>
        </timecode>
        <media>
            <video>
                <track>
"""
timeline_end="""                    <enabled>TRUE</enabled>
                    <locked>FALSE</locked>
                </track>
                <format>
                    <samplecharacteristics>
                        <width>1920</width>
                        <height>1080</height>
                        <pixelaspectratio>square</pixelaspectratio>
                        <rate>
                            <timebase>60</timebase>
                            <ntsc>FALSE</ntsc>
                        </rate>
                        <codec>
                            <appspecificdata>
                                <appname>Final Cut Pro</appname>
                                <appmanufacturer>Apple Inc.</appmanufacturer>
                                <data>
                                    <qtcodec/>
                                </data>
                            </appspecificdata>
                        </codec>
                    </samplecharacteristics>
                </format>
            </video>
            <audio>
                <track>
                    <enabled>TRUE</enabled>
                    <locked>FALSE</locked>
                </track>
            </audio>
        </media>
    </sequence>
</xmeml>
"""

clipitem_mid="""
        <timecode>
            <string>00:00:00:00</string>
            <displayformat>NDF</displayformat>
            <rate>
                <timebase>60</timebase>
                <ntsc>FALSE</ntsc>
            </rate>
        </timecode>
        <media>
            <video>
                <duration>1</duration>
                <samplecharacteristics>
                    <width>1920</width>
                    <height>1080</height>
                </samplecharacteristics>
            </video>
        </media>
    </file>
    <compositemode>normal</compositemode>
    <filter>
        <enabled>TRUE</enabled>
        <start>0</start>
        <end>3600001</end>
        <effect>
            <name>Basic Motion</name>
            <effectid>basic</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>Scale</name>
                <parameterid>scale</parameterid>
                <value>100</value>
                <keyframe>
                    <when>216000</when>
                    <value>85</value>
                </keyframe>
                <keyframe>
                    <when>216015</when>
                    <value>100</value>
                    <inbez>
                        <horiz>-3</horiz>
                        <vert>0</vert>
                    </inbez>
                </keyframe>
                <valuemin>0</valuemin>
                <valuemax>10000</valuemax>
            </parameter>
            <parameter>
                <name>Center</name>
                <parameterid>center</parameterid>
                <value>
                    <horiz>0</horiz>
                    <vert>0</vert>
                </value>
            </parameter>
            <parameter>
                <name>Rotation</name>
                <parameterid>rotation</parameterid>
                <value>0</value>
                <valuemin>-100000</valuemin>
                <valuemax>100000</valuemax>
            </parameter>
            <parameter>
                <name>Anchor Point</name>
                <parameterid>centerOffset</parameterid>
                <value>
                    <horiz>0</horiz>
                    <vert>0</vert>
                </value>
            </parameter>
        </effect>
    </filter>
    <filter>
        <enabled>TRUE</enabled>
        <start>0</start>
        <end>3600001</end>
        <effect>
            <name>Crop</name>
            <effectid>crop</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>left</name>
                <parameterid>left</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>right</name>
                <parameterid>right</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>top</name>
                <parameterid>top</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>bottom</name>
                <parameterid>bottom</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
        </effect>
    </filter>"""


clipitem_end="""    <filter>
        <enabled>TRUE</enabled>
        <start>-1</start>
        <end>-1</end>
        <effect>
            <name>Time Remap</name>
            <effectid>timeremap</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>speed</name>
                <parameterid>speed</parameterid>
                <value>0</value>
                <valuemin>-10000</valuemin>
                <valuemax>10000</valuemax>
            </parameter>
            <parameter>
                <name>reverse</name>
                <parameterid>reverse</parameterid>
                <value>FALSE</value>
            </parameter>
            <parameter>
                <name>frameblending</name>
                <parameterid>frameblending</parameterid>
                <value>FALSE</value>
            </parameter>
            <parameter>
                <name>variablespeed</name>
                <parameterid>variablespeed</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>1</valuemax>
            </parameter>
            <parameter>
                <name>graphdict</name>
                <parameterid>graphdict</parameterid>
                <keyframe>
                    <when>0</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfstart>TRUE</speedkfstart>
                </keyframe>
                <keyframe>
                    <when>216000</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfin>TRUE</speedkfin>
                </keyframe>
                <keyframe>
                    <when>216013</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfout>TRUE</speedkfout>
                </keyframe>
                <keyframe>
                    <when>3600001</when>
                    <value>1</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfend>TRUE</speedkfend>
                </keyframe>
                <valuemin>0</valuemin>
                <valuemax>0</valuemax>
                <interpolation>
                    <name>FCPCurve</name>
                </interpolation>
            </parameter>
        </effect>
    </filter>
</clipitem>
"""

In [27]:
timeline=timeline_start
for i,filename in enumerate(filenames):
    start = 30*i
    end = start+30
    clipitem_start = f"""<clipitem id="{filename} 0">
    <name>{filename}</name>
    <duration>3600001</duration>
    <rate>
        <timebase>60</timebase>
        <ntsc>FALSE</ntsc>
    </rate>
    <start>{start}</start>
    <end>{end}</end>
    <enabled>TRUE</enabled>
    <in>{216000}</in>
    <out>{216000+end-start}</out>
    <file id="{filename} 2">
        <duration>1</duration>
        <rate>
            <timebase>60</timebase>
            <ntsc>FALSE</ntsc>
        </rate>
        <name>{filename}</name>
        <pathurl>file://localhost/{path}/{filename}</pathurl>"""   
    clipitem_opacity=f"""
                        <filter>
                            <enabled>TRUE</enabled>
                            <start>0</start>
                            <end>3600001</end>
                            <effect>
                                <name>Opacity</name>
                                <effectid>opacity</effectid>
                                <effecttype>motion</effecttype>
                                <mediatype>video</mediatype>
                                <effectcategory>motion</effectcategory>
                                <parameter>
                                    <name>opacity</name>
                                    <parameterid>opacity</parameterid>
                                    <value>100</value>
                                    <keyframe>
                                        <when>{216000+end-1}</when>
                                        <value>100</value>
                                    </keyframe>
                                    <keyframe>
                                        <when>{216000+end}</when>
                                        <value>0</value>
                                    </keyframe>
                                    <valuemin>0</valuemin>
                                    <valuemax>100</valuemax>
                                </parameter>
                            </effect>
                        </filter>""" 
    timeline+=clipitem_start + clipitem_mid + clipitem_opacity+ clipitem_end
timeline+=timeline_end

In [28]:
with open('Timeline'+date.today().strftime('%d%b%y')+'.xml','w') as f:
    f.write(timeline)